In [22]:
#
#
#Author: WU Nan
#Date: 2016.12.5
#Description: This is the csv to csv programme file to 
#Further target: 
#1) should calulate "whether it is Normal Distribution or not "  
#2) Give data visualization
#Give the very normal risk ranking
#
#
#
#
from __future__ import print_function
from elasticsearch import Elasticsearch
import sys
from operator import add
from pyspark import SparkContext
from pyspark.mllib.regression import LinearRegressionWithSGD, LabeledPoint
from pyspark.mllib.classification import SVMWithSGD, SVMModel

import numpy as np
import csv
import math
from scipy import stats
import pandas as pd
import scipy


#period: the days of period you choose
#✅WELL DONE
def get_csv_data(filename):
    filename = "file:/Users/nancywu/sparkhadoop/datatest/largetest/"+filename+".csv"
    File = sc.textFile(filename)
    File.map(lambda line: line.split(","))
    File.filter(lambda line: len(line) > 0)
    File.map(lambda line: (line[0], line[1]))
    data = File.collect()
    stock_text = [d.split(",") for d in data]

    #From backward of Period of present date (90 days before 2016.4.1) to present date 2016.4.1
    #historical period you want to choose
    period = 365*3

    open_price = [round(float(stock_text[i][1]),10) for i in range(period,0,-1)]
    close_price = [round(float(stock_text[i][4]),10) for i in range(period,0,-1)]
    volume=[long(stock_text[i][5]) for i in range (period, 0,-1)]
    date=["Date"]
    [date.append(stock_text[i][0]) for i in range(period,0,-1)]

    print ("get_csv_data: done")
    return open_price,close_price,volume,date


#NUMPY WRONG!!solved it already on Nov.28   ✅

def risk_assessment_price_change (filename):
    open_price,close_price, volume, date= get_csv_data(filename)
    
    ##risk distance round精度调整
    risk_assessment_initial = list(map(lambda x: round((x[0]-x[1]),2), zip(open_price, close_price)))
    ##
    print("====risk assessment initial======")
    tmp=sc.parallelize(risk_assessment_initial)
    Change_initial = tmp.map(lambda distance: (distance, 1))\
            .reduceByKey(lambda a, b: a + b)
    Change_list=Change_initial.collect()
    distance_list=[]
    frequency_list=[]
    for (distance,frequency) in Change_list:
        #这里限制了波动噪声。
        if (frequency<len(risk_assessment_initial)*0.5):
            distance_list.append(distance)
            frequency_list.append(frequency)
    
    print ("==========Change list=======================")
    #print(Change_list)
    return Change_list,distance_list,frequency_list

def normal_distribution_test(frequency_list):
    
    #3 statistics to test how distribution it is.
    #Make risk range for particular stock
        x = frequency_list
        shapiro_results = scipy.stats.shapiro(x)
        a1=shapiro_results[0]
        a2=round(shapiro_results[1],8)
        matrix_sw = [
                ['', 'DF', 'Test Statistic_SW', 'p-value'],
                ['Sample Data', len(x) - 1, a1, a2]
                ]
        print(matrix_sw)
                #shapiro_result more close to 1 more probabily of normal distribution it is
                #####
        ks_results = scipy.stats.kstest(x, cdf='norm')
        b1=ks_results[0]
        b2=round(ks_results[1],8)
        matrix_ks = [
                ['', 'DF', 'Test Statistic_KS', 'p-value'],
                ['Sample Data', len(x) - 1, b1, b2]
                ]
        
        print(matrix_ks)
                ###
        dagostino_results = scipy.stats.mstats.normaltest(x)
        c1=dagostino_results[0]
        c2=round(dagostino_results[1],8)
        matrix_dp = [
                ['', 'DF', 'Test Statistic_dp', 'p-value'],
                ['Sample Data', len(x) - 1, c1, c2]
                ]
        print(matrix_dp)
        
        if (abs(a1-a2)>0.8 and (abs(b1-b2)>0.8 and b2<0.0005) and (abs(c1-c2)>0.8 and c2<0.0005)):
            #normal distribution. low risk
            signal=3
        elif (abs(a1-a2)>1 or abs(b1-b2)>1 or abs(c1-c2)>1): 
            signal=2
        else:
            #not normal distribution. high risk
            signal =1
        return signal 
    
# Haven't done yet
def risk_range(risk_signal,name):
    #Date=
    if (risk_signal==3):
                print ("==========Low Risk Stock===========:==",name)
                assessment=[name,'Low Risk']
    if (risk_signal==2):
                print ("==========Netural Risk Stock===========:==",name)
                assessment=[name,'Netural Risk']
    if (risk_signal==1):
                print ("==========High Risk Stock===========:==",name) 
                assessment=[name,'High Risk']
    
    
    return assessment

#Do not  test it yet⚠️
def writeToElastic(fileindex,es,filename,stock_text):
    df=stock_text
    j = 1
    actions = []
    count = int(len(df))
    while (j < count):
        action = {
                   "_index": fileindex, # 这里不可以是大写，都是小写
                   "_type": filename,
                   "_id": j,
                   "_source": {
                               "date":df[j][0],
                               "open":float(df[j][1]),
                               "high":float(df[j][2]),
                               "low":float(df[j][3]),
                               "close":float(df[j][4]),
                               "volume":int(df[j][5]),
                               "adjClose":float(df[j][6]),
                               #"timestamp": datetime.now()
                                }
                   }
        print(action)
        actions.append(action)
        j += 1
        if (len(actions) == 180):
            helpers.bulk(es, actions)
            del actions[0:len(actions)]
            
    if (len(actions) >0 ):
            helpers.bulk(es, actions)
            del actions[0:len(actions)]
    

    
    
        
if __name__ =="__main__":
    #sc = SparkContext(appName="Monte Carlo")
    Ticker = sc.textFile("file:/Users/nancywu/sparkhadoop/datatest/Tickertest.csv")
    filelist = Ticker.map(lambda f: f.split(",")).collect()
    #l = Ticker.collect()
    #filelist = l[0].split(",")
    print(filelist)
    es = Elasticsearch()
    risk_list=[]
    print("===========start============")
    for f in filelist:
        try:
            name = f[0]+".csv"
            print (name,"simulation",f[0])
            change_list,distance_list,frequency_list = risk_assessment_price_change(f[0])
            #alist=[["distance"]+distance_list,["freq"]+frequency_list]
            alist=pd.DataFrame(change_list,columns=['Price Change','Frequence'])
            print("=====risk assessment pass=====")
            print(len(change_list))
            #blist=pd.read_csv('file:/Users/nancywu/sparkhadoop/datatest/largetest/AA.csv')
            #print(ggplot(alist,aes('Price Change','Frequence'))+geom_point()+stat_smooth(colour='blue', span=0.2))
            #pl.plot(distance_list,frequency_list ,'ro')
            #pl.xlabel("Price Change")
            #pl.ylabel("Frequency")
            #pl.title(f[0])
            #pl.show()
            risk_signal=normal_distribution_test(frequency_list)
            print("=======output pass=======")
            risk_list.append(risk_range(risk_signal, f[0]))
            ########
            sc.parallelize(change_list).repartition(1).saveAsTextFile("file:/Users/nancywu/sparkhadoop/datatest_result/" + 'risk_change_list'+name)
            #writeToElastic("predictvalue",es,name,output)
        except:
            import traceback
            traceback.print_exc()
            print("No service for this stock")
    sc.parallelize(risk_list).repartition(1).saveAsTextFile("file:/Users/nancywu/sparkhadoop/datatest_result/stock_risk_list")




[[u'AA'], [u'AAPL'], [u'ZWELCAST.BO'], [u'ZYA.DU'], [u'ZYA.MU'], [u'ZYA.SG'], [u'ZYB.BE'], [u'ZZZ.L'], [u'600000.SS']]
===========start============
AA.csv simulation AA
get_csv_data: done
====risk assessment initial======
==========Change list=======================
=====risk assessment pass=====
112
[['', 'DF', 'Test Statistic_SW', 'p-value'], ['Sample Data', 111, 0.8173558712005615, 0.0]]
[['', 'DF', 'Test Statistic_KS', 'p-value'], ['Sample Data', 111, 0.84134474606854293, 0.0]]
[['', 'DF', 'Test Statistic_dp', 'p-value'], ['Sample Data', 111, 17.103638345751705, 0.00019319]]
=======output pass=======
==========Low Risk Stock===========:== AA
No service for this stock
AAPL.csv simulation AAPL
get_csv_data: done
====risk assessment initial======
==========Change list=======================

Traceback (most recent call last):
  File "<ipython-input-22-1a737ff00889>", line 206, in <module>
    sc.parallelize(change_list).repartition(1).saveAsTextFile("file:/Users/nancywu/sparkhadoop/datatest_result/" + 'risk_change_list'+name)
  File "/Users/nancywu/sparkhadoop/python/pyspark/rdd.py", line 1506, in saveAsTextFile
    keyed._jrdd.map(self.ctx._jvm.BytesToString()).saveAsTextFile(path)
  File "/Users/nancywu/sparkhadoop/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 813, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/nancywu/sparkhadoop/python/pyspark/sql/utils.py", line 45, in deco
    return f(*a, **kw)
  File "/Users/nancywu/sparkhadoop/python/lib/py4j-0.9-src.zip/py4j/protocol.py", line 308, in get_return_value
    format(target_id, ".", name), value)
Py4JJavaError: An error occurred while calling o5757.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/Users/nancywu/sparkhadoop/da


=====risk assessment pass=====
762
[['', 'DF', 'Test Statistic_SW', 'p-value'], ['Sample Data', 761, 0.6097652316093445, 0.0]]
[['', 'DF', 'Test Statistic_KS', 'p-value'], ['Sample Data', 761, 0.84134474606854293, 0.0]]
[['', 'DF', 'Test Statistic_dp', 'p-value'], ['Sample Data', 761, 316.67864975646677, 0.0]]
=======output pass=======
==========Netural Risk Stock===========:== AAPL
No service for this stock

  File "<ipython-input-22-1a737ff00889>", line 206, in <module>
    sc.parallelize(change_list).repartition(1).saveAsTextFile("file:/Users/nancywu/sparkhadoop/datatest_result/" + 'risk_change_list'+name)
  File "/Users/nancywu/sparkhadoop/python/pyspark/rdd.py", line 1506, in saveAsTextFile
    keyed._jrdd.map(self.ctx._jvm.BytesToString()).saveAsTextFile(path)
  File "/Users/nancywu/sparkhadoop/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 813, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/nancywu/sparkhadoop/python/pyspark/sql/utils.py", line 45, in deco
    return f(*a, **kw)
  File "/Users/nancywu/sparkhadoop/python/lib/py4j-0.9-src.zip/py4j/protocol.py", line 308, in get_return_value
    format(target_id, ".", name), value)
Py4JJavaError: An error occurred while calling o5836.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/Users/nancywu/sparkhadoop/datatest_result/risk_change_listAAPL.


ZWELCAST.BO.csv simulation ZWELCAST.BO
No service for this stock
ZYA.DU.csv simulation ZYA.DU
No service for this stock
ZYA.MU.csv simulation ZYA.MU
get_csv_data: done
====risk assessment initial======
==========Change list=======================

...

  File "<ipython-input-22-1a737ff00889>", line 206, in <module>
    sc.parallelize(change_list).repartition(1).saveAsTextFile("file:/Users/nancywu/sparkhadoop/datatest_result/" + 'risk_change_list'+name)
  File "/Users/nancywu/sparkhadoop/python/pyspark/rdd.py", line 1506, in saveAsTextFile
    keyed._jrdd.map(self.ctx._jvm.BytesToString()).saveAsTextFile(path)
  File "/Users/nancywu/sparkhadoop/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 813, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/nancywu/sparkhadoop/python/pyspark/sql/utils.py", line 45, in deco
    return f(*a, **kw)
  File "/Users/nancywu/sparkhadoop/python/lib/py4j-0.9-src.zip/py4j/protocol.py", line 308, in get_return_value
    format(target_id, ".", name), value)
Py4JJavaError: An error occurred while calling o5954.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/Users/nancywu/sparkhadoop/datatest_result/risk_change_listZYA.M


=====risk assessment pass=====
208
[['', 'DF', 'Test Statistic_SW', 'p-value'], ['Sample Data', 206, 0.7942911386489868, 0.0]]
[['', 'DF', 'Test Statistic_KS', 'p-value'], ['Sample Data', 206, 0.84134474606854293, 0.0]]
[['', 'DF', 'Test Statistic_dp', 'p-value'], ['Sample Data', 206, 58.71491355002113, 0.0]]
=======output pass=======
==========Netural Risk Stock===========:== ZYA.SG
No service for this stock
ZYB.BE.csv simulation ZYB.BE
No service for this stock
ZZZ.L.csv simulation ZZZ.L
No service for this stock
600000.SS.csv simulation 600000.SS
get_csv_data: done
====risk assessment initial======
==========Change list=======================

  File "<ipython-input-22-1a737ff00889>", line 190, in <module>
    change_list,distance_list,frequency_list = risk_assessment_price_change(f[0])
  File "<ipython-input-22-1a737ff00889>", line 58, in risk_assessment_price_change
    open_price,close_price, volume, date= get_csv_data(filename)
  File "<ipython-input-22-1a737ff00889>", line 45, in get_csv_data
    open_price = [round(float(stock_text[i][1]),10) for i in range(period,0,-1)]
IndexError: list index out of range
Traceback (most recent call last):
  File "<ipython-input-22-1a737ff00889>", line 190, in <module>
    change_list,distance_list,frequency_list = risk_assessment_price_change(f[0])
  File "<ipython-input-22-1a737ff00889>", line 58, in risk_assessment_price_change
    open_price,close_price, volume, date= get_csv_data(filename)
  File "<ipython-input-22-1a737ff00889>", line 38, in get_csv_data
    data = File.collect()
  File "/Users/nancywu/sparkhadoop/python/pyspark/rdd.py", line 771, in collect
    port = self.ctx.


=====risk assessment pass=====
148
[['', 'DF', 'Test Statistic_SW', 'p-value'], ['Sample Data', 147, 0.4235299825668335, 0.0]]
[['', 'DF', 'Test Statistic_KS', 'p-value'], ['Sample Data', 147, 0.84134474606854293, 0.0]]
[['', 'DF', 'Test Statistic_dp', 'p-value'], ['Sample Data', 147, 237.40201549527393, 0.0]]
=======output pass=======
==========Netural Risk Stock===========:== 600000.SS
No service for this stock


  File "<ipython-input-22-1a737ff00889>", line 206, in <module>
    sc.parallelize(change_list).repartition(1).saveAsTextFile("file:/Users/nancywu/sparkhadoop/datatest_result/" + 'risk_change_list'+name)
  File "/Users/nancywu/sparkhadoop/python/pyspark/rdd.py", line 1506, in saveAsTextFile
    keyed._jrdd.map(self.ctx._jvm.BytesToString()).saveAsTextFile(path)
  File "/Users/nancywu/sparkhadoop/python/lib/py4j-0.9-src.zip/py4j/java_gateway.py", line 813, in __call__
    answer, self.gateway_client, self.target_id, self.name)
  File "/Users/nancywu/sparkhadoop/python/pyspark/sql/utils.py", line 45, in deco
    return f(*a, **kw)
  File "/Users/nancywu/sparkhadoop/python/lib/py4j-0.9-src.zip/py4j/protocol.py", line 308, in get_return_value
    format(target_id, ".", name), value)
Py4JJavaError: An error occurred while calling o6151.saveAsTextFile.
: org.apache.hadoop.mapred.FileAlreadyExistsException: Output directory file:/Users/nancywu/sparkhadoop/datatest_result/risk_change_list60000

In [6]:
x=[1,1,1,1]
y=[2,2,2,2]
alist=[["distance","freq"],x,y]
blist=pd.read_csv('file:/Users/nancywu/sparkhadoop/datatest/largetest/AA.csv')
alist=pd.DataFrame(alist,columns=['AA','FREQ','D','DD'])
print (alist)

         AA  FREQ    D   DD
0  distance  freq  NaN  NaN
1         1     1  1.0  1.0
2         2     2  2.0  2.0
